In [1]:
import re
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [2]:
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters and digits
    tokens = word_tokenize(text)  # Tokenize text
    stop_words = set(stopwords.words('english'))  # Load stopwords
    tokens = [t for t in tokens if t not in stop_words]  # Remove stopwords
    return ' '.join(tokens)

In [3]:
def load_and_prepare_data(filepath):
    df = pd.read_csv(filepath)
    df['processed_text'] = df['text'].apply(preprocess_text)  # Apply preprocessing
    X = df['processed_text']  # Features (text)
    y = df['label']  # Target labels (1 = Real, 0 = Fake)
    return train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [4]:
def train_model_naive_bayes(dataset_path):
    global vectorizer, model  # Make them accessible for prediction later
    
    # Load data
    X_train, X_test, y_train, y_test = load_and_prepare_data(dataset_path)

    # Transform text data using TF-IDF
    vectorizer = TfidfVectorizer(max_features=5000)
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)

    # Train Naive Bayes Model
    model = MultinomialNB()
    model.fit(X_train_tfidf, y_train)

    # Evaluate
    y_pred = model.predict(X_test_tfidf)
    print("🔹 Model Accuracy:", accuracy_score(y_test, y_pred))
    print("🔹 Classification Report:\n", classification_report(y_test, y_pred))

In [5]:
def predict_news(text):
    # Preprocess new text
    processed_text = preprocess_text(text)
    # Transform using the trained TF-IDF vectorizer
    text_tfidf = vectorizer.transform([processed_text])
    # Predict
    prediction = model.predict(text_tfidf)[0]
    probability = model.predict_proba(text_tfidf)[0]

    return {
        'prediction': 'REAL' if prediction == 1 else 'FAKE',
        'confidence': probability.max()
    }

In [6]:
dataset_path = r"C:\Users\msi\Desktop\NLP\Dataset\Generated_Fakenews_dataset.csv"

In [7]:
train_model_naive_bayes(dataset_path)

🔹 Model Accuracy: 1.0
🔹 Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       900
           1       1.00      1.00      1.00       900

    accuracy                           1.00      1800
   macro avg       1.00      1.00      1.00      1800
weighted avg       1.00      1.00      1.00      1800



In [8]:
sample_text = "Breaking: Scientists discover new planet that's made entirely of gold!"
result = predict_news(sample_text)
print(f"Prediction: {result['prediction']}")
print(f"Confidence: {result['confidence']:.2%}")

Prediction: FAKE
Confidence: 98.40%


In [10]:
sample_text = "The Moroccan government has solid leaders"
result = predict_news(sample_text)
print(f"Prediction: {result['prediction']}")
print(f"Confidence: {result['confidence']:.2%}")

Prediction: REAL
Confidence: 56.95%
